# Chatbot with BLUE score

__결과__
  - 학습이 수렴하지 않아 대답을 생성하지 못한다.
  - 이유는 아직 파악이 안 됨

In [89]:
import os
import re
import random

import numpy as np
import pandas as pd
import tensorflow as tf
import nltk
import gensim
# from gensim.models import KeyedVectors
from konlpy.tag import Mecab
from tqdm.notebook import tqdm
from nltk.translate.bleu_score import sentence_bleu
from nltk.translate.bleu_score import SmoothingFunction

print(np.__version__)
print(pd.__version__)
print(tf.__version__)
print(nltk.__version__)
print(gensim.__version__)

1.21.4
1.3.3
2.6.0
3.6.5
3.8.3


In [90]:
os.system("mkdir -p ~/aiffel/gd_12/data/")
os.system("ln -s ~/data/* ~/aiffel/gd_12/data/")

ln: failed to create symbolic link '/aiffel/aiffel/gd_12/data/ChatbotData.csv': File exists
ln: failed to create symbolic link '/aiffel/aiffel/gd_12/data/ko.bin': File exists
ln: failed to create symbolic link '/aiffel/aiffel/gd_12/data/word2vec_ko.model': File exists
ln: failed to create symbolic link '/aiffel/aiffel/gd_12/data/word2vec_ko.model.trainables.syn1neg.npy': File exists
ln: failed to create symbolic link '/aiffel/aiffel/gd_12/data/word2vec_ko.model.wv.vectors.npy': File exists


256

In [91]:
d_path = "/aiffel/aiffel/gd_12/data/"
raw_data = pd.read_csv(d_path+"ChatbotData.csv")
raw_data.head()

,Q,A,label
0,12시 땡!,하루가 또 가네요.,0
1,1지망 학교 떨어졌어,위로해 드립니다.,0
2,3박4일 놀러가고 싶다,여행은 언제나 좋죠.,0
3,3박4일 정도 놀러가고 싶다,여행은 언제나 좋죠.,0
4,PPL 심하네,눈살이 찌푸려지죠.,0


In [92]:
def preprocess_sentence(sentence):
    
    # 모든 입력을 소문자로 변환; 알파벳, 문장부호, 한글만 남기고 모두 제거
    sentence = re.sub(r"[^0-9a-zA-Z가-힣ㄱ-ㅎㅏ-ㅣ\?.!,]+", " ", sentence.lower())
        
    # 문장 앞뒤의 불필요한 공백을 제거합니다
    sentence = sentence.strip()
    sentence = re.sub(r'[" "]+', " ", sentence)

    return sentence

In [93]:
questions = [str(s) for s in  raw_data["Q"]]
answers = [str(s) for s in  raw_data["A"]]
print('전체 샘플 수 :', len(questions))
print('전체 샘플 수 :', len(answers))

전체 샘플 수 : 11823
전체 샘플 수 : 11823


__데이터 전처리 및 토큰화__

In [94]:
'''
Src, Tgt 한 쪽의 중복도 허용하지 않는 경우 예를들어,
("A", "B")
("A", "D")
("C", "D")
위 경우 ("A", "D") 쌍이 먼저 포함되면
("A", "B")나 ("C", "D") 모두 사용할 수 없다.
'''
def build_corpus(src_raw, tgt_raw, max_len):
    # 문장 전처리
    src_corpus = [preprocess_sentence(s) for s in  src_raw]
    tgt_corpus = [preprocess_sentence(s) for s in  tgt_raw]
    
    # 토큰화
    src_corpus = [mecab_split(s) for s in  src_corpus]
    tgt_corpus = [mecab_split(s) for s in  tgt_corpus]
    
    # 긴 문장 제외
    filtered_idcs = [i for i, s in enumerate(zip(src_corpus, tgt_corpus))
                             if len(s[0])<=max_len and len(s[1])<=max_len]
    
    # 중복문 제외
    src_result, tgt_result = list(), list()
    src_instnts, tgt_instnts = dict(), dict()
    for i in filtered_idcs:
        s = src_instnts.setdefault(src_corpus[i], []); s.append(i)
        t = tgt_instnts.setdefault(tgt_corpus[i], []); t.append(i)
        if len(t)>1: continue
        if len(s)>1: continue
        src_result.append(src_corpus[i])
        tgt_result.append(tgt_corpus[i])
    
    return src_result, tgt_result

In [95]:
mecab = Mecab()

def mecab_split(sentence):
    return tuple(mecab.morphs(sentence))

In [96]:
que_corpus , ans_corpus = build_corpus(questions, answers, max_len=30)

In [97]:
print('전처리 샘플 수 :', len(que_corpus))
print('전처리 샘플 수 :', len(ans_corpus))

전처리 샘플 수 : 7646
전처리 샘플 수 : 7646


In [98]:
wv_path = "/aiffel/aiffel/gd_12/"
model_dir = os.path.join(wv_path, 'ko.bin')

# from gensim.models.keyedvectors import Word2VecKeyedVectors

# wv = Word2VecKeyedVectors.load(model_dir)

from gensim.models import Word2Vec
ko_w2v = Word2Vec.load(model_dir)
# model = Word2Vec.load_word2vec_format(model_dir, binary=True)

# with open(model_dir, 'rb') as f:
#     wv = Word2VecKeyedVectors.load_word2vec_format(
#             f, binary=True, unicode_errors='ignore',) # , limit=500000

In [99]:
ko_w2v.wv.most_similar("성실")

[('정직', 0.7132465839385986),
 ('겸손', 0.6951979398727417),
 ('완고', 0.6625800728797913),
 ('근면', 0.6611404418945312),
 ('친절', 0.6598716974258423),
 ('냉정', 0.6536936163902283),
 ('솔직', 0.6535051465034485),
 ('현명', 0.6468929648399353),
 ('충실', 0.6405693292617798),
 ('공평', 0.6393522620201111)]

__데이터 증강__

In [100]:
def lexical_sub(tokens, wv):
    # tokens = sentence.split()

    selected_tok = random.randrange(len(tokens))

    result = []
    for idx, tok in enumerate(tokens):
        if idx is selected_tok:
            try:
                result.append(wv.most_similar(tok)[0][0])
            # 로드된 word2vec 모델에 단어가 없을 경우
            except KeyError as E:
                return None
 
        else:
            result.append(tok)

    return tuple(result)

In [101]:
new_src_tgt = ([], [])
random.seed(25)

for old_src, old_tgt in tqdm(zip(que_corpus, ans_corpus)):
    new_src = lexical_sub(old_src, ko_w2v.wv)
    new_tgt = lexical_sub(old_tgt, ko_w2v.wv)
    if new_src is not None: 
        new_src_tgt[0].append(new_src)
        new_src_tgt[1].append(old_tgt)
    if new_tgt is not None: 
        new_src_tgt[0].append(old_src)
        new_src_tgt[1].append(new_tgt)

0it [00:00, ?it/s]

In [102]:
print(len(new_src_tgt[0]), len(new_src_tgt[1]))

for i in range(20):
    print(new_src_tgt[0][i], new_src_tgt[1][i])

13334 13334
('12', '시', '땡', '캐치') ('하루', '가', '또', '가', '네요', '.')
('12', '시', '땡', '!') ('일주일', '가', '또', '가', '네요', '.')
('1', '중퇴', '학교', '떨어졌', '어') ('위로', '해', '드립니다', '.')
('3', '박', '4', '일', '놀', '러', '가', '기에', '싶', '다') ('여행', '은', '언제나', '좋', '죠', '.')
('3', '박', '4', '일', '놀', '러', '가', '고', '싶', '다') ('항해', '은', '언제나', '좋', '죠', '.')
('ppl', '강하', '네') ('눈살', '이', '찌푸려', '지', '죠', '.')
('sns', '들어맞', '팔', '왜', '안', '하', '지', 'ㅠㅠ') ('잘', '모르', '고', '있', '을', '수', '도', '있', '어요', '.')
('sns', '맞', '팔', '왜', '안', '하', '지', 'ㅠㅠ') ('잘', '모르', '고', '있', '을', '수', '도', '있', '어요', '는데')
('sns', '분간', '낭비', '인', '거', '아', '는데', '매일', '하', '는', '중') ('시간', '을', '정하', '고', '해', '보', '세요', '.')
('sns', '시간', '낭비', '인', '거', '아', '는데', '매일', '하', '는', '중') ('시간', '을', '정하', '기에', '해', '보', '세요', '.')
('sns', '보', '면', '나', '만', '빼', '고', '다', '사랑', '해', '보여') ('자랑', '하', '는', '자리', '니까요', '.')
('sns', '보', '면', '나', '만', '빼', '고', '다', '행복', '해', '보여') ('자랑', '하', '는', '자리', 'ㄴ데요', '.

In [144]:
enc_corpus = que_corpus + new_src_tgt[0]
dec_corpus = ans_corpus + new_src_tgt[1]
dec_corpus = [ ["<bos>"]+list(s)+["<eos>"] for s in dec_corpus ]

min_len = min(len(s) for s in enc_corpus+dec_corpus)
max_len = max(len(s) for s in enc_corpus+dec_corpus)

print("min_len: ", min_len)
print("max_len: ", max_len)

min_len:  1
max_len:  32


In [145]:
min_len = 3
max_len = 30

q_list, a_list = [], []

for q, a in zip(enc_corpus, dec_corpus):
    if len(q)<min_len: continue
    if len(a)<min_len: continue   
    if len(q)>max_len: continue
    if len(a)>max_len: continue
    q_list.append(q); a_list.append(a)

enc_corpus, dec_corpus = q_list, a_list

In [146]:
print(len(enc_corpus), len(dec_corpus))

20260 20260


In [148]:
print(enc_corpus[0])

('12', '시', '땡', '!')


__훈련-검증 분리__

In [149]:
from sklearn.model_selection import train_test_split
enc_train, enc_val, dec_train, dec_val = train_test_split(
                                enc_corpus, dec_corpus, test_size=0.05, random_state=42)

In [150]:
print(enc_train[0])

('나', '를', '좋아하', '는', '애', '랑', '친구', '가능', '?')


__토크나이저 사전 생성__

In [151]:
ko_tokenizer = tf.keras.preprocessing.text.Tokenizer(
                                filters='', lower=False, oov_token='')
ko_tokenizer.fit_on_texts(list(s) for s in enc_train+dec_train)

In [152]:
print("Vocab Size:", VOCAB_SIZE:=len(ko_tokenizer.index_word))
# mecab_tokenizer.sequences_to_texts([mecab_tensor[100]])
# tensor = tokenizer.texts_to_sequences(corpus)
ko_tokenizer.sequences_to_texts([[i] for i in range(10)])

Vocab Size: 6950


['', '', '.', '<bos>', '<eos>', '이', '는', '하', '을', '가']

In [153]:
ko_tokenizer.texts_to_sequences([["<unk>"]])

[[1]]

In [154]:
ko_tokenizer.texts_to_sequences([["<uuuunk>"]])

[[1]]

In [155]:
ko_tokenizer.texts_to_sequences([[""]])

[[1]]

__시퀀스 패딩__

In [156]:
MAX_LEN = 30

def padded_seqs(corpus, tokenizer):  # corpus: Tokenized Sentence's List
    
    if isinstance(corpus[0], list):
        tensor = tokenizer.texts_to_sequences(corpus)
    else: 
        tensor = tokenizer.texts_to_sequences(list(s) for s in corpus)
    
    tensor = tf.keras.preprocessing.sequence.pad_sequences(
                                tensor, maxlen=MAX_LEN, padding='post')
    return tensor

In [157]:
enc_train = padded_seqs(enc_train, ko_tokenizer)
enc_val = padded_seqs(enc_val, ko_tokenizer)
dec_train = padded_seqs(dec_train, ko_tokenizer)
dec_val = padded_seqs(dec_val, ko_tokenizer)

In [158]:
print(enc_train[:3])
print(dec_train[:3])

[[  22   32  170    6   94  134   45  211   18    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0]
 [  96  560    2    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0]
 [1251    9    6   24  574   20   18    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0]]
[[   3  820   90   70   36   33   49    2    4    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0]
 [   3 1343   41  369    9   14   64   22  171    2    4    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0]
 [   3 5922    8 5923  259   19   12    2    4    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0]]


In [159]:
BATCH_SIZE = 128 # 64

train_dataset = tf.data.Dataset.from_tensor_slices(
            (enc_train, dec_train) ).batch(batch_size=BATCH_SIZE)

In [160]:
# Positional Encoding 구현
def positional_encoding(pos, d_model):
    def cal_angle(position, i):
        return position / np.power(10000, int(i) / d_model)

    def get_posi_angle_vec(position):
        return [cal_angle(position, i) for i in range(d_model)]

    sinusoid_table = np.array([get_posi_angle_vec(pos_i) for pos_i in range(pos)])
    sinusoid_table[:, 0::2] = np.sin(sinusoid_table[:, 0::2])
    sinusoid_table[:, 1::2] = np.cos(sinusoid_table[:, 1::2])
    
    return tf.cast(sinusoid_table , tf.float32) 

In [161]:
def generate_padding_mask(seq):
    seq = tf.cast(tf.math.equal(seq, 0), tf.float32)
    return seq[:, tf.newaxis, tf.newaxis, :]

def generate_causality_mask(size):
    mask = 1 - tf.linalg.band_part(tf.ones((size, size)), -1, 0)
    return mask  # (seq_len, seq_len)

def generate_masks(src, tgt):
    enc_mask = generate_padding_mask(src)
    dec_enc_mask = generate_padding_mask(src)
    
    dec_padd_mask = generate_padding_mask(tgt)
    causality_mask = generate_causality_mask(tgt.shape[1])
    combined_mask = tf.maximum(dec_padd_mask, causality_mask) ###

    return enc_mask, combined_mask, dec_enc_mask

In [162]:
class MultiHeadAttention(tf.keras.layers.Layer):
    def __init__(self, d_model, num_heads):
        super(MultiHeadAttention, self).__init__()
        self.num_heads = num_heads
        self.d_model = d_model
        
        self.depth = d_model // self.num_heads
        
        self.W_q = tf.keras.layers.Dense(d_model) # Linear Layer
        self.W_k = tf.keras.layers.Dense(d_model)
        self.W_v = tf.keras.layers.Dense(d_model)
        
        self.linear = tf.keras.layers.Dense(d_model)

    def scaled_dot_product_attention(self, Q, K, V, mask):
        d_k = tf.cast(K.shape[-1], tf.float32)
        
        QK = tf.matmul(Q, K, transpose_b=True) # LMS 예시
        scaled_qk = QK / tf.sqrt(d_k)
        
        if mask is not None: scaled_qk += (mask * -1e9)
       
        attentions = tf.nn.softmax(scaled_qk, axis=-1) # 예시 참고
        out = tf.matmul(attentions, V)
        
        return out, attentions
        
    def split_heads(self, x):
    
        seq_len = x.shape[1]
        split_x = tf.reshape(x,
                         [-1, seq_len, self.num_heads, self.depth] )
        split_x = tf.transpose(split_x, perm=[0, 2, 1, 3])
        
        return split_x

    def combine_heads(self, x):
  
        seq_len = x.shape[2] # 예시 참고
        combined_x = tf.transpose(x, perm=[0, 2, 1, 3])
        combined_x = tf.reshape(combined_x,
                         [-1, seq_len, self.d_model] )
        
        return combined_x
    
    def call(self, Q, K, V, mask):
  
        WQ = self.W_q(Q)
        WK = self.W_q(K)
        WV = self.W_q(V)
        
        WQ_split = self.split_heads(WQ) # 예시 참고
        WK_split = self.split_heads(WK)
        WV_split = self.split_heads(WV)
        
        out, attention_weights = self.scaled_dot_product_attention(
                                    WQ_split, WK_split, WV_split, mask )
        out = self.combine_heads(out)
        out = self.linear(out)
        
        return out, attention_weights

In [163]:
class PoswiseFeedForwardNet(tf.keras.layers.Layer):
    def __init__(self, d_model, d_ff):
        super(PoswiseFeedForwardNet, self).__init__()
        self.w_1 = tf.keras.layers.Dense(d_ff, activation='relu')
        self.w_2 = tf.keras.layers.Dense(d_model)

    def call(self, x):
        out = self.w_1(x)
        out = self.w_2(out)
            
        return out

In [164]:
class EncoderLayer(tf.keras.layers.Layer):
    def __init__(self, d_model, n_heads, d_ff, dropout):
        super(EncoderLayer, self).__init__()

        self.enc_self_attn = MultiHeadAttention(d_model, n_heads)
        self.ffn = PoswiseFeedForwardNet(d_model, d_ff)

        self.norm_1 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
        self.norm_2 = tf.keras.layers.LayerNormalization(epsilon=1e-6)

        self.dropout = tf.keras.layers.Dropout(dropout)
        
    def call(self, x, mask, training):

        residual = x
        out = self.norm_1(x)
        out, enc_attn = self.enc_self_attn(out, out, out, mask)
        out = self.dropout(out, training=training)
        out += residual
        
        residual = out
        out = self.norm_2(out)
        out = self.ffn(out)
        out = self.dropout(out, training=training)
        out += residual
        
        return out, enc_attn

In [165]:
# DecoderLayer 클래스를 작성하세요.
class DecoderLayer(tf.keras.layers.Layer):
    def __init__(self, d_model, n_heads, d_ff, dropout):
        super(DecoderLayer, self).__init__()

        self.dec_self_attn = MultiHeadAttention(d_model, n_heads)
        self.enc_dec_attn = MultiHeadAttention(d_model, n_heads)
        self.ffn = PoswiseFeedForwardNet(d_model, d_ff)

        self.norm_1 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
        self.norm_2_1 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
        self.norm_2_2 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
        self.norm_3 = tf.keras.layers.LayerNormalization(epsilon=1e-6)

        self.dropout = tf.keras.layers.Dropout(dropout)
        
    def call(self, x1, x2, c_mask, mask, training):

        residual = x1
        norm_x1 = self.norm_1(x1)
        out, dec_attn = self.dec_self_attn(norm_x1, norm_x1, norm_x1, c_mask) ###
        out = self.dropout(out, training=training)
        out += residual
 
        residual = out
        norm_x1 = self.norm_2_1(out)
        norm_x2 = self.norm_2_2(x2) 
        out, dec_enc_attn = self.enc_dec_attn(norm_x1, norm_x2, norm_x2, mask)
        out = self.dropout(out, training=training)
        out += residual
        
        residual = out
        out = self.norm_3(out)
        out = self.ffn(out)
        out = self.dropout(out, training=training)
        out += residual
        
        return out, dec_attn, dec_enc_attn

In [166]:
class Encoder(tf.keras.Model):
    def __init__(self,
                 n_layers,
                 d_model,
                 n_heads,
                 d_ff,
                 dropout):
        super(Encoder, self).__init__()
        self.n_layers = n_layers
        self.enc_layers = [EncoderLayer(d_model, n_heads, d_ff, dropout) 
                        for _ in range(n_layers)]
        
    def call(self, x, mask, training):
        out = x
    
        enc_attns = list()
        for i in range(self.n_layers):
            out, enc_attn = self.enc_layers[i](out, mask, training)
            enc_attns.append(enc_attn)
        
        return out, enc_attns

In [167]:
class Decoder(tf.keras.Model):
    def __init__(self,
                 n_layers,
                 d_model,
                 n_heads,
                 d_ff,
                 dropout):
        super(Decoder, self).__init__()
        self.n_layers = n_layers
        self.dec_layers = [DecoderLayer(d_model, n_heads, d_ff, dropout) 
                            for _ in range(n_layers)]
                            
                            
    def call(self, x, enc_out, causality_mask, padding_mask, training):
        out = x
    
        dec_attns = list()
        dec_enc_attns = list()
        for i in range(self.n_layers):
            out, dec_attn, dec_enc_attn = \
              self.dec_layers[i](out, enc_out, causality_mask, padding_mask, training)
            dec_attns.append(dec_attn)
            dec_enc_attns.append(dec_enc_attn)

        return out, dec_attns, dec_enc_attns

In [168]:
class Transformer(tf.keras.Model):
    def __init__(self,
                 n_layers,
                 d_model,
                 n_heads,
                 d_ff,
                 src_vocab_size,
                 tgt_vocab_size,
                 pos_len,
                 dropout=0.2,
                 shared=True,
                 shared_emb=False,):
        super(Transformer, self).__init__()
        
        self.d_model = tf.cast(d_model, tf.float32)
        self.shared = shared # 예시 참고
        self.shared_emb = shared_emb # 예시 참고
        
        if shared_emb:
            self.src_embedding = self.tgt_embedding = \
                        tf.keras.layers.Embedding(src_vocab_size, d_model)
        else:
            self.src_embedding = tf.keras.layers.Embedding(src_vocab_size, d_model)
            self.tgt_embedding = tf.keras.layers.Embedding(tgt_vocab_size, d_model)
        
        self.pos_encoding = positional_encoding(pos_len, d_model) 
        
        self.encoder = Encoder(n_layers, d_model, n_heads, d_ff, dropout)
        self.decoder = Decoder(n_layers, d_model, n_heads, d_ff, dropout)
        
        self.linear = tf.keras.layers.Dense(tgt_vocab_size) # 예시 참고
        
        if shared : # 예시 참고
            self.shared_weights = self.tgt_embedding.weights
            self.linear.set_weights(tf.transpose(self.shared_weights))
        
        # self.dropout = tf.keras.layers.Dropout(dropout)

    def embedding(self, emb, x, shared=False):
  
        seq_len = x.shape[1]
        out = emb(x)
        if shared: out *= tf.math.sqrt(self.d_model)
        pos_enc = self.pos_encoding[np.newaxis, ...][:, :seq_len, :]
        out = out + pos_enc
        
        return out

    def call(self, enc_in, dec_in, enc_mask, combined_mask, dec_enc_mask, training):
        
        if self.shared_emb: enc_shared = self.shared
        else: enc_shared = False
        
        enc_in = self.embedding(self.src_embedding, enc_in, enc_shared)
        dec_in = self.embedding(self.tgt_embedding, dec_in, self.shared)
        
        enc_out, enc_attns = self.encoder(enc_in, enc_mask, training)
        dec_out, dec_attns, dec_enc_attns = self.decoder(
                            dec_in, enc_out, combined_mask, dec_enc_mask, training)
        
        logits = self.linear(dec_out)
        
        return logits, enc_attns, dec_attns, dec_enc_attns

In [169]:
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True, reduction='none')

def loss_function(real, pred):
    # 실제 시퀀스중 0인 값을 0, 나머지 1로 반환
    mask = tf.math.logical_not(tf.math.equal(real, 0))
    loss_ = loss_object(real, pred)

    mask = tf.cast(mask, dtype=loss_.dtype)
    # 실제 시퀀스중 0값을 갖는 위치의 손실에 0을 곱해줌
    loss_ *= mask

    # Masking 되지 않은 입력의 개수로 Scaling하는 과정
    return tf.reduce_sum(loss_)/tf.reduce_sum(mask)

In [170]:
class LearningRateScheduler(tf.keras.optimizers.schedules.LearningRateSchedule):
    def __init__(self, d_model, warmup_steps=4000):
        super(LearningRateScheduler, self).__init__()
        self.d_model = d_model
        self.warmup_steps = warmup_steps
    
    def __call__(self, step):
        arg1 = step ** -0.5
        arg2 = step * (self.warmup_steps ** -1.5)
        
        return (self.d_model ** -0.5) * tf.math.minimum(arg1, arg2)

In [212]:
@tf.function()
def train_step(src, tgt, model, optimizer):
    gold = tgt[:, 1:]
        
    enc_mask, combined_mask, dec_enc_mask = generate_masks(src, tgt)

    with tf.GradientTape() as tape:
        predictions, enc_attns, dec_attns, dec_enc_attns = \
                model(src, tgt, enc_mask, combined_mask, dec_enc_mask, training=True)
        loss = loss_function(gold, predictions[:, :-1])


    variables = model.trainable_variables
    gradients = tape.gradient(loss, variables)
    optimizer.apply_gradients(zip(gradients, variables))
    
    return loss, enc_attns, combined_mask, dec_enc_attns

@tf.function  # 예제코드 재사용
def eval_step(src, tgt, model):
    gold = tgt[:, 1:]
        
    enc_mask, combined_mask, dec_enc_mask = generate_masks(src, tgt)
    
    predictions, enc_attns, dec_attns, dec_enc_attns = \
            model(src, tgt, enc_mask, combined_mask, dec_enc_mask, training=False)
    
    loss = loss_function(gold, predictions[:, :-1])

    return loss

In [213]:
BATCH_SIZE = 64 # 64

train_dataset = tf.data.Dataset.from_tensor_slices(
            (enc_train, dec_train) ).batch(batch_size=BATCH_SIZE)

N_LAYERS = 6
D_MODEL = 256
N_HEADS = 8
D_FF = 2048
pos_len = MAX_LEN # 40
DROPOUT = 0.1

transformer = Transformer(
    n_layers=N_LAYERS,
    d_model=D_MODEL,
    n_heads=N_HEADS,
    d_ff=D_FF,
    src_vocab_size=VOCAB_SIZE,
    tgt_vocab_size=VOCAB_SIZE,
    pos_len=pos_len,
    dropout=DROPOUT,
    shared=True,
    shared_emb=True)

In [214]:
learning_rate = LearningRateScheduler(D_MODEL, warmup_steps=1000)
optimizer = tf.keras.optimizers.Adam(#0.00001)
                    learning_rate,beta_1=0.9,beta_2=0.98, epsilon=1e-9)
# optimizer = tf.keras.optimizers.RMSprop(0.00001)
#                         learning_rate)

In [215]:
sequence_len = 30

sample_enc = tf.random.uniform((BATCH_SIZE, sequence_len))
sample_dec = tf.random.uniform((BATCH_SIZE, sequence_len+7))

sample_enc = tf.keras.preprocessing.sequence.pad_sequences(sample_enc, padding='post', maxlen=MAX_LEN)
sample_dec = tf.keras.preprocessing.sequence.pad_sequences(sample_dec, padding='post', maxlen=MAX_LEN)

enc_mask, combined_mask, dec_enc_mask = \
    generate_masks(sample_enc, sample_dec)

sample_logits, enc_attns, dec_attns, dec_enc_attns = transformer(
        sample_enc, sample_dec, enc_mask, combined_mask, dec_enc_mask )

print ('enc_mask:', enc_mask.shape) # .shape
print ('dec_enc_mask:', dec_enc_mask.shape) # .shape
print ('dec_mask:', combined_mask.shape) # .shape
print ('enc_attns:', enc_attns[0].shape) # .shape

print ('Decoder Output:', sample_logits.shape)
print ('dec_attns:', dec_attns[0].shape) # .shape
print ('dec_enc_attns:', dec_enc_attns[0].shape) # .shape

transformer.summary()

enc_mask: (64, 1, 1, 30)
dec_enc_mask: (64, 1, 1, 30)
dec_mask: (64, 1, 30, 30)
enc_attns: (64, 8, 30, 30)
Decoder Output: (64, 30, 6950)
dec_attns: (64, 8, 30, 30)
dec_enc_attns: (64, 8, 30, 30)
Model: "transformer_10"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_17 (Embedding)     multiple                  1779200   
_________________________________________________________________
encoder_10 (Encoder)         multiple                  7100928   
_________________________________________________________________
decoder_10 (Decoder)         multiple                  7896576   
_________________________________________________________________
dense_938 (Dense)            multiple                  1786150   
Total params: 18,562,854
Trainable params: 18,562,854
Non-trainable params: 0
_________________________________________________________________


In [216]:
def gen_answer(tokens, model, src_tokenizer, tgt_tokenizer):
    '''
    translate와 동일
    '''
    padded_tokens = tf.keras.preprocessing.sequence.pad_sequences(
                            [tokens], maxlen=MAX_LEN, padding='post' )
    ids = []
    # output = tf.expand_dims([tgt_tokenizer.bos_id()], 0)   
    output = tf.expand_dims([tgt_tokenizer.word_index["<bos>"]], 0)   
    for i in range(MAX_LEN):
        enc_padding_mask, combined_mask, dec_padding_mask = \
                                        generate_masks(padded_tokens, output)

        predictions, _, _, _ = model(padded_tokens, 
                                      output,
                                      enc_padding_mask,
                                      combined_mask,
                                      dec_padding_mask)

        predicted_id = \
        tf.argmax(tf.math.softmax(predictions, axis=-1)[0, -1]).numpy().item()

        # if tgt_tokenizer.eos_id() == predicted_id:
        if tgt_tokenizer.word_index["<eos>"] == predicted_id:
            # result = tgt_tokenizer.decode_ids(ids)  
            result = tgt_tokenizer.sequences_to_texts([ids])  
            return result[0]

        ids.append(predicted_id)
        output = tf.concat([output, tf.expand_dims([predicted_id], 0)], axis=-1)

    # result = tgt_tokenizer.decode_ids(ids)
    result = tgt_tokenizer.sequences_to_texts([ids])  
    
    return result[0]

In [217]:
# print(dataset_count)
run_idx = 0

In [218]:
assert not np.any(np.isnan(enc_train))

In [220]:
EPOCHS = 3
for epoch in range(EPOCHS):
    
    total_loss = 0
    
    idx_list = list(range(0, len(enc_train), BATCH_SIZE)) # 
    random.shuffle(idx_list)
    
    t = tqdm(idx_list)
    t.set_description_str('Epoch %2d' % (epoch + 1))

    for (batch, idx) in enumerate(t):
        if len(enc_train[idx:idx+BATCH_SIZE]) < BATCH_SIZE: continue
        batch_loss, _, _, _ = train_step(enc_train[idx:idx+BATCH_SIZE], ##
                                dec_train[idx:idx+BATCH_SIZE],
                                transformer,
                                optimizer)
    
        total_loss += batch_loss
        # print(batch_loss.numpy())
        # t.set_postfix_str('Loss %.4f' % float(batch_loss.numpy() ) )
        t.set_postfix_str('Loss %.4f' % (total_loss.numpy() / (batch + 1)))

    if (epoch+1) % 5 == 0: pass
    elif epoch+1 == EPOCHS: pass
    else: continue
    
    test_loss = 0
    
    idx_list = list(range(0, len(enc_val), BATCH_SIZE)) # 
    random.shuffle(idx_list)
    t = tqdm(idx_list)
    t.set_description_str('Test Epoch %2d' % (epoch + 1))
    
    for (test_batch, idx) in enumerate(t):
        test_batch_loss = eval_step(enc_val[idx:idx+BATCH_SIZE],
                                    dec_val[idx:idx+BATCH_SIZE],
                                    transformer,)
    
        test_loss += test_batch_loss
        t.set_postfix_str('Test Loss %.4f' % (test_loss.numpy() / (test_batch + 1)))
    
    for example in enc_val[:16:4]:
        print("src: ", ko_tokenizer.sequences_to_texts([example])[0])
        tgt = gen_answer(example, transformer, ko_tokenizer, ko_tokenizer)
        print("tgt: ", tgt)

  0%|          | 0/301 [00:00<?, ?it/s]

  0%|          | 0/301 [00:00<?, ?it/s]

  0%|          | 0/301 [00:00<?, ?it/s]

  0%|          | 0/16 [00:00<?, ?it/s]

src:  지하철 만 타 면 생각나                         
tgt:                               
src:  너 누구 ...                           
tgt:                               
src:  6 개월 간 인의 짧 은 사랑 .                      
tgt:                               
src:  완전 재미 없 거든                          
tgt:                               


In [ ]:
print(len(enc_train[idx:idx+BATCH_SIZE]), batch, idx, BATCH_SIZE)

for s in enc_train[idx:idx+BATCH_SIZE]:
    print(s)
    print(ko_tokenizer.sequences_to_texts([s])[0])

In [ ]:
def eval_bleu_single(model, src_sentence, tgt_sentence, src_tokenizer, tgt_tokenizer, verbose=True):
    
    if isinstance(src_sentence[0], np.int32):
        src_tokens = src_sentence
        tgt_tokens = tgt_sentence
        src_sentence = src_tokenizer.sequences_to_texts([src_tokens])[0]
        tgt_sentence = tgt_tokenizer.sequences_to_texts([tgt_tokens])[0]
    else:
        src_tokens = src_tokenizer.texts_to_sequences([src_sentence])[0]
        tgt_tokens = tgt_tokenizer.texts_to_sequences([tgt_sentence])[0]

    if (len(src_tokens) > MAX_LEN): return None
    if (len(tgt_tokens) > MAX_LEN): return None

    reference = tgt_sentence.split()
    candidate = gen_answer(src_tokens, model, src_tokenizer, tgt_tokenizer).split()

    score = sentence_bleu([reference], candidate,
                          smoothing_function=SmoothingFunction().method1)

    if verbose:
        print("Source Sentence: ", src_sentence)
        print("Model Prediction: ", candidate)
        print("Real: ", reference)
        print("Score: %lf\n" % score)
        
    return score

In [ ]:
def eval_bleu(model, src_sentences, tgt_sentence, src_tokenizer, tgt_tokenizer, verbose=True):
    total_score = 0.0
    sample_size = len(src_sentences)
    
    for idx in tqdm(range(sample_size)):
        score = eval_bleu_single(model, src_sentences[idx], tgt_sentence[idx],
                                 src_tokenizer, tgt_tokenizer, verbose)
        if not score: continue
        
        total_score += score
    
    print("Num of Sample:", sample_size)
    print("Total Score:", total_score / sample_size)

In [ ]:
def calculate_bleu(reference, candidate, weights=[0.25, 0.25, 0.25, 0.25]):
    return sentence_bleu([reference],
                         candidate,
                         weights=weights,
                         smoothing_function=SmoothingFunction().method1)  # smoothing_function 적용


In [ ]:
# test_idx = 0
for test_idx in range(0,100,20):
    eval_bleu_single(transformer, 
                     enc_val[test_idx], 
                     dec_val[test_idx], 
                     ko_tokenizer, 
                     ko_tokenizer)